**Initial setup notebook setup**



In [1]:
import requests
import httpimport
import pandas as pd
from os import path
# this imports the code from github.
if path.exists("zp_tools.py"):
  import zp_tools as zpt
else:
  with httpimport.remote_repo(['zp_tools'], 'https://raw.githubusercontent.com/Heteroskedastic/Zwiftpower_tools/master'):
    import zp_tools as zpt

# Configuration
team_id = "2740" # CRYO-GEN
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}


Get a list of team members and save CSV with Division and name.

In [2]:
# This outputs a list of members ansd there Div
s = requests.Session()
team = zpt.get_team_data(team_id, headers=headers, s=s)
team['URL'] = 'https://www.zwiftpower.com/profile.php?z=' + str(team.zwid)
#team_selected = team[['div', 'divw', 'flag', 'name', 'zwid', 'URL']].copy()
team_selected = team.copy()
team_selected.sort_values(by=['div', 'divw'], ascending=False, inplace=True)
team_selected['div_letter'] = team_selected['div'].replace({40:'D', 30:'C', 20:'B', 10:'A',  5:'A+',  0:'Z'})
team_selected['divw_letter'] = team_selected['divw'].replace({40:'D', 30:'C', 20:'B', 10:'A',  5:'A+',  0:'Z'})
team_selected[['div_letter', 'divw_letter', 'flag', 'name', 'zwid', 'URL']].head(10)
team_f = team_selected[team_selected.divw.notnull()]
team_selected.to_csv('team.csv')

In [3]:
team_f = team_selected[team_selected.divw.notnull()]
team_f['ftp'] = team_f.ftp[0]

ValueError: Length of values does not match length of index

**Work to get event data**

In [ ]:
#t_results = get_team_results_data(team_id, header=headers, s=s)

# def get_event_results_data(event_id):
#     # https://www.zwiftpower.com/api3.php?do=event_results&zid={team_id}
#     #    return pd.DataFrame(s.get(url, headers=headers).json()['data'])
def get_team_results_page(event_id):
    url = f'https://www.zwiftpower.com/events.php?zid={event_id}'
    return BeautifulSoup(s.get(url, headers=headers).content, 'html.parser')

def get_event_name(event_id):
    page = get_team_results_page(event_id)
    event_info = page.find("div", {"class": "col-sm-7"})
    return {'info':event_info, 'name':event_info.find("h3").text, 'date':event_info.find(id="EVENT_DATE").text}

ei = get_event_name('292676')
    
#t_results.head(10)

In [2]:
zpt.get_team_data(2740).to_csv('team_riders.csv')

In [3]:
z = zpt.get_team_data(2740)

In [34]:
zwift = z.copy()
zwift= zwift.drop(['flag', 'r', 'ftp', 'aid', 'age', 'zada', 'w',
       'rank', 'skill', 'distance', 'climbed', 'energy', 'time', 'skill_race',
       'skill_seg', 'skill_power', 'status', 'reg', 'email',
       'h_1200_watts', 'h_1200_wkg', 'h_15_watts', 'h_15_wkg'], axis=1)

original = pd.read_csv('data/Original_kit.csv')
kit = pd.read_csv('data/kit_yes_no.csv')

zwift.rename(columns={'zwid':'ZWID', 'name':'NAME'}, inplace=True)
original['ZWEMAIL'] = original.ZWEMAIL.str.lower().str.strip()
kit['ZWEMAIL'] = kit.ZWEMAIL.str.lower().str.strip()
original['NAME'] = original.NAME.str.lower().str.strip()
kit['NAME'] = kit.NAME.str.lower().str.strip()

original.drop_duplicates('ZWEMAIL', inplace=True)
original.drop_duplicates('ZWID', inplace=True)
kit.drop_duplicates('ZWEMAIL', inplace=True)

zwift = zwift.merge(original, on='ZWID', how='left')
zwift.sort_values('ZWID', inplace=True)
print(f'Number of ZWID: {zwift.ZWID.count()}\nNumber of unique ZWID: {zwift.ZWID.nunique()}')
zwift.drop_duplicates(['ZWEMAIL', 'ZWID'], inplace=True)
print(f'After DeDup \n Number of ZWID: {zwift.ZWID.count()}\n Number of unique ZWID: {zwift.ZWID.nunique()}')
zwift = zwift.merge(kit, on='ZWEMAIL', how='outer')
zwift.sort_values('ZWID', inplace=True)
zwift.drop_duplicates()
zwift.rename(columns={'NAME_x':'ZP_NAME', 'NAME':'NAME_KIT', 'NAME_y':'ZWIFT_NAME', 'KIT':'KIT_STATUS'}, inplace=True)
zwift.loc[zwift.KIT_STATUS.isnull() & zwift.ZWEMAIL.notnull(), 'KIT_STATUS'] = 'Requested'
zwift.loc[(zwift.KIT_STATUS == 'No') & (zwift.ZWEMAIL.notnull()), 'KIT_STATUS'] = 'Requested'
zwift.loc[zwift.ZWEMAIL.isnull(), 'KIT_STATUS'] = 'Need to complete form'


zwift['div'].replace({10:'A', 20:'B', 30:'C', 40:'D', 0:''}, inplace=True)
zwift.divw.replace({10:'A', 20:'B', 30:'C', 40:'D', 0:''}, inplace=True)
zwift['Partial Email'] = zwift.ZWEMAIL.str.slice(stop=3)

zwift[['ZP_NAME', 'ZWIFT_NAME', 'ZWID', 'div', 'divw', 'Partial Email', 'KIT_STATUS']].to_csv('data/public list.csv')

# zwift = zwift[zwift.KIT != 'Yes']
zwift.sort_values('ZWID', inplace=True)
#zwift[zwift.ZWEMAIL.notnull()].head(200)
zwift.tail(50)


Number of ZWID: 563
Number of unique ZWID: 563
After DeDup 
 Number of ZWID: 563
 Number of unique ZWID: 563


,div,divw,ZP_NAME,ZWID,ZWIFT_NAME,ZPA,ZWEMAIL,KIT_STATUS,NAME_KIT,Partial Email
539,C,,Nick Holyome CRYO-GEN (C),1321229.0,nick holyome dirt c,Yes,nholyome@gmail.com,Yes,nick holyome cryo-gen (c),nho
186,B,,Eric Innes,1321282.0,NaN,NaN,NaN,Need to complete form,NaN,NaN
187,C,,Andrew Hellinga (CRYO-GEN),1321836.0,NaN,NaN,NaN,Need to complete form,NaN,NaN
540,C,,Euan Stronach [Cryo-Gen],1325407.0,euan stronach,Yes,euan.stronach@gmail.com,Requested,NaN,eua
541,D,,Eddie Pizza Cryo,1332005.0,eddie davidson,Yes,eddiedavidson08@gmail.com,Requested,NaN,edd
542,B,A,Angela MacRae (CRYO-GEN Ospreys),1332978.0,angela macrae,Yes,angela.macrae@outlook.com.au,Requested,NaN,ang
543,D,,Zach Coates CRYO-GEN,1337846.0,zach coates,Yes,zachcoates@live.co.uk,Requested,NaN,zac
188,C,,Joel Chua,1340348.0,NaN,NaN,NaN,Need to complete form,NaN,NaN
189,,,David Peach,1345630.0,NaN,NaN,NaN,Need to complete form,NaN,NaN
544,C,B,Beth Maciver CRYO-GEN,1351393.0,beth maciver [cryo-gen],Yes,bethmaciver21@gmail.com,Yes,beth maciver cryo-gen,bet


In [6]:
clean = zwift[(zwift.ZWEMAIL.notnull()) & (zwift.KIT != 'Yes') & (zwift.ZWID.notnull()) & (zwift.ZWWIFT_NAME.notnull())].copy()
clean['Partial Email'] = clean.ZWEMAIL.str.slice(stop=3)
clean[['ZP_NAME', 'ZWIFT_NAME', 'NAME_KIT', 'Partial Email']].to_csv('clean.csv')

AttributeError: 'DataFrame' object has no attribute 'NAME_ORG'

In [ ]:
clean[clean.NAME_ORG.notnull()][['NAME_ZP', 'NAME_ORG', 'NAME_KIT']]

In [4]:
z.columns

Index(['div', 'divw', 'flag', 'r', 'ftp', 'name', 'aid', 'age', 'zada', 'w',
       'rank', 'skill', 'distance', 'climbed', 'energy', 'time', 'skill_race',
       'skill_seg', 'skill_power', 'zwid', 'status', 'reg', 'email',
       'h_1200_watts', 'h_1200_wkg', 'h_15_watts', 'h_15_wkg'],
      dtype='object')

In [ ]:
'Qinyin Zhou (CRYO-GEN)	'.strip()

In [22]:
zwift.KIT_STATUS.unique()

array(['Requested'], dtype=object)